### Implementation of classification of MNIST dataset using Bayesian Neural Network (BNN)

In [1]:
import torchbnn as bnn
import torch
import numpy as np
import keras
import os
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train: np.ndarray = x_train.astype("float32") / 255
x_test: np.ndarray = x_test.astype("float32") / 255
x_train = x_train.reshape(x_train.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)

# x_train = np.expand_dims(x_train, -1)
# x_test = np.expand_dims(x_test, -1)

print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

x_train shape: (60000, 784)
y_train shape: (60000,)
60000 train samples
10000 test samples


In [ ]:
device = ('cpu')

class BayesianMLP(torch.nn.Module):
    def __init__(self):
        super(BayesianMLP, self).__init__()
        self.hid1 = bnn.BayesLinear(
            prior_mu=0,
            prior_sigma=0.1,
            in_features=28 * 28,
            out_features=20
        )
        self.out = bnn.BayesLinear(
            prior_mu=0,
            prior_sigma=0.1,
            in_features=20,
            out_features=10
        )

    def forward(self, x):
        z = torch.relu(self.hid1(x))
        z = self.out(z)
        return z

In [3]:
test_model = BayesianMLP()
test_model.eval()
_ = torch.rand(3, 28*28)
print(test_model(_).shape)

torch.Size([3, 10])


In [4]:
x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

batch_size = 16
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model = BayesianMLP().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 10
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)

        optimizer.zero_grad()
        output = model(batch_x)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}")

model.eval()
correct = 0
total = 0

with torch.no_grad():
    for batch_x, batch_y in test_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        outputs = model(batch_x)
        predictions = torch.argmax(outputs, dim=1)
        correct += (predictions == batch_y).sum().item()
        total += batch_y.size(0)

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


Epoch [1/10], Loss: 0.5656
Epoch [2/10], Loss: 0.2800
Epoch [3/10], Loss: 0.2184
Epoch [4/10], Loss: 0.1879
Epoch [5/10], Loss: 0.1640
Epoch [6/10], Loss: 0.1493
Epoch [7/10], Loss: 0.1399
Epoch [8/10], Loss: 0.1294
Epoch [9/10], Loss: 0.1227
Epoch [10/10], Loss: 0.1163
Test Accuracy: 95.84%
